<a href="https://colab.research.google.com/github/AnastTF/Regression_laser_Speed/blob/main/Regression_laserSpeed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#импортируем библиотеки pandas, numpy и matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_excel("/content/Зависимости для АМг3М 1мм_1.xls")#открытие файла с названием "Зависимости для АМг3М 1мм_1.xls" и импортируем данные из этого файла 
#print(data.shape)#количество колонок и столбцов таблицы
#print(data.head())#выводит первые 5 строк таблицы

#подготовка данных
#извлекаем данные
x=data.iloc[:,:-1].values#столбец количество кадров
y=data.iloc[:,1].values#столбец значений рабочей скорости режущего инструмента для соответствующего значения количества кадров

#вычислим коэффициент корреляции
coeffKorr=data['Количество кадров'].corr(data['Значение рабочей скорости, м/с'])

#разделяем данные на тестовые и обучающие
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4,random_state=0)#разбиваем данные на 80% - обучающие, 20% - тестовые

#'обучаем' алгоритм с использованием метода наименьших квадратов - МНК
from sklearn.linear_model import LinearRegression
model=LinearRegression()#вычисляем оптимальные значения весов
z=model.fit(np.log(x_train),y_train)
b = z.intercept_#значение коэффициента b аппроксимирующей функции y=a*ln(x)+b
a = z.coef_#значение коэффициента a аппроксимирующей функции y=a*ln(x)+b
# Для сравнения качества делаем прогноз и на тех данных, на которых обучались
z_train = model.predict(np.log(x_train))
z_test = model.predict(np.log(x_test))
#делаем прогнозы по модели по тестовым данным
y_pred=model.predict(np.log(x_test))
#сравниваем фактические выходные значения для x_test с прогнозируемыми значениями
df=pd.DataFrame({'Actual':y_test,'Predicted': y_pred})

#оценка алгоритма
from sklearn import metrics
#найдем коэффициент детерминации
r_sq=model.score(np.log(x_train),y_train)
#найдем значение средней абсолютной ошибки-MAE(Mean Abcolute Error)
m_a_e=metrics.mean_absolute_error(y_test,y_pred)
#найдем значение средней квадратической ошибки - MSE (Mean Squared Error)
m_s_e=metrics.mean_squared_error(y_test,y_pred)
#найдем значение среднеквадратичной ошибки - RMSE (Root Mean Squared Error)
r_m_s_e=np.sqrt(metrics.mean_squared_error(y_test,y_pred))

#значения для построения регрессирующей функции на графике
x_pr=[]
for row in x_train:
    for i in row:
        x_pr.append(i)
x_pr.sort()
x_pr=np.array(x_pr).reshape((-1,1))#переводим в двумерный массив
z_pr = model.predict(np.log(x_pr))

lab=str(round((*z.coef_),3))+' *ln(x)+ '+str(round(z.intercept_,3))
#ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ на графике с использованием библиотеки matplotlib    
plt.plot(x_train, y_train,color='green',marker='o',ls='')#marker-маркер точек, ls - стиль линии
plt.plot(x_test, y_test,color='blue',marker='v',ls='')
plt.plot(x_pr, z_pr,color='red',label=lab)
plt.title("Зависимость скорости режущего инструмента"+"\n"+"на рабочем ходе от числа кадров для АМг3М, 1мм")#ВВЕДИТЕ название в соответствии с вашими данными
plt.xlabel("Количество кадров УП")
plt.ylabel("Von(факт),м/с")
plt.grid()
plt.legend()

#Выводим результаты
   
print('='*20)
print("coeffKorr:"+' '*(len('Коэффициент детерминации:')-5)+'|',coeffKorr)
print('Коэффициент детерминации:'+' '*5+'|',r_sq)
print('Mean Abcolute Error:'+' '*10+'|',m_a_e)
print('Mean Squared Error:'+' '*11+'|',m_s_e)
print('Root Mean Squared Error:'+' '*6+'|',r_m_s_e)
print('='*20)
print('Сравнение фактических выходных значений \nдля тестовых данных с прогнозируемыми значениями: ')
print(df)
print('='*20)


# Новый раздел